In [28]:
import nltk, os, gensim
import numpy as np
import pandas as pd

In [33]:
claimDF = pd.read_csv("IBM_EACL-2017.v1/claim_stance_dataset_v1.csv")
print claimDF[:1].T
with open('IBM_EACL-2017.v1/articles/t1/clean_1.txt') as f:
    test = f.read()
    print test[418: 568]

                                                                                0
topicId                                                                         1
split                                                                        test
topicText                       This house believes that the sale of violent v...
topicTarget                             the sale of violent video games to minors
topicSentiment                                                                 -1
claims.claimId                                                               2973
claims.stance                                                                 PRO
claims.claimCorrectedText       Exposure to violent video games causes at leas...
claims.claimOriginalText        exposure to violent video games causes at leas...
claims.article.rawFile                                      articles/t1/raw_1.txt
claims.article.rawSpan.start                                                  490
claims.article.r

In [8]:
# functions from https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [24]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model


w2vec = get_word2vec(MySentences([test]), 'w2vmodel')

Found w2vmodel


In [18]:
all_docs = []
for root, dirs, files in os.walk("IBM_EACL-2017.v1"):
    for file in files:
        if 'clean' in file:
            file_name = os.path.join(root, file)
            with open(file_name) as f:
                text = f.read().decode('utf-8')
                all_docs.append(text)
                

In [23]:
print len(all_docs)
for i, a in enumerate(all_docs):
    print i, len(a)
get_word2vec(MySentences(all_docs), 'w2vmodel')

1056
0 25582
1 43193
2 20594
3 6954
4 34794
5 14431
6 2447
7 3932
8 3911
9 7213
10 8490
11 16114
12 351
13 63843
14 592
15 26686
16 34234
17 20862
18 3350
19 5061
20 15146
21 2970
22 9156
23 8627
24 27380
25 31865
26 17472
27 22603
28 23804
29 29089
30 8053
31 34537
32 76129
33 21984
34 2490
35 3485
36 2769
37 22592
38 6124
39 9217
40 35499
41 20215
42 7432
43 8982
44 12406
45 13137
46 2039
47 28232
48 6165
49 38148
50 11411
51 32230
52 43748
53 52700
54 22290
55 6866
56 33395
57 7742
58 29629
59 21707
60 12285
61 30380
62 21356
63 18977
64 5670
65 32454
66 7108
67 14857
68 26295
69 42955
70 1743
71 21405
72 27272
73 53397
74 16917
75 12405
76 8012
77 34439
78 64367
79 46892
80 51937
81 16656
82 8563
83 36101
84 49420
85 976
86 27767
87 3922
88 12470
89 3833
90 15226
91 3644
92 6006
93 70172
94 24512
95 45671
96 4266
97 56948
98 26824
99 68673
100 5037
101 8125
102 23591
103 19467
104 15273
105 12064
106 4470
107 37616
108 19965
109 5895
110 22822
111 6164
112 22198
113 31929
114 24621

Model done training. Saving to disk


In [35]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
print mean_embedding_vectorizer
mean_embedded = mean_embedding_vectorizer.fit_transform(test)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [37]:
print len(mean_embedded)
mean_embedded[0]

43457


array([ 0.31426018, -1.06831205,  1.21346712,  1.86929476, -0.71056241,
       -0.21541671,  0.72629952, -0.73208243, -1.36275637, -1.22459245,
       -1.21996188, -0.54398781, -0.35856432, -0.14329609, -0.15196806,
       -0.9756965 , -1.21144581, -0.91734874,  1.0777564 ,  0.02624179,
       -1.60658813,  0.18764953, -1.81937814,  1.91089845,  0.58192432,
        1.65468061, -1.71986306, -0.43812519, -0.92625415, -0.48045486,
       -1.16743231,  0.43014014,  0.65899193, -0.27752998,  1.65941036,
        0.79171795, -0.13793954,  0.57609439,  2.16613722, -0.19841804,
        2.05687428,  0.98856151,  0.59611279, -0.11774249,  0.80156952,
        1.83594668,  0.90304452, -1.20070708,  0.26908618,  0.75146681,
        0.63377053,  1.01785088, -0.36910939,  2.1938746 ,  0.11118388,
       -0.68866092, -1.12002301,  1.05376661,  0.9628073 , -0.63872081,
       -1.48452842, -1.09537327,  2.41032434, -1.23613763,  0.64009857,
        0.03496607, -1.71336448,  0.13561393, -0.45263085, -0.94

In [34]:
print test

. 

Controversies over video games often center on topics such as video game graphic violence, sex and sexism, violent and gory scenes, partial or full nudity, portrayal of criminal behavior, racism, and other provocative and objectionable material. 

Video games have been studied for links to addiction and aggression. Earlier meta-analyses (an analysis of several studies) were conflicting. A 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world. A decrease in prosocial behavior (caring about the welfare and rights of others) was also noted [REF]. Another 2001 meta-analyses using similar methods[REF] and a more recent 2009 study focusing specifically on serious aggressive behavior[REF] concluded that video game violence is not related to serious aggressive behavior in real life. 

Many potential positive effects have been proposed [REF]. Recent research has suggested t